In [1]:
url = 'https://adventofcode.com/2020/day/20/input'
from helper import getdata
t = getdata(url,False)
import numpy as np
from itertools import product

## 1

In [162]:
def transform(t):
    tiles = {}
    for tile in filter(lambda x : x != '', t.split('\n\n')):
        title, square = tile.split('\n', 1)
        tiles[int(title[5:-1])] = (
            np.array([list(line) for line in square.splitlines()])=='#'
        ).astype(int)
    return tiles


def generator(tiles, next_rest):
    for tk in next_rest:
        for flud, rot in list(product([False, True],[0,1,2,3])):
            tile = tiles[tk]
            if flud:
                tile = np.flipud(tile)
            tile = np.rot90(tile, rot)
            edges = {}
            edges['n'] = tile[0]
            edges['s'] = tile[-1]
            edges['e'] = tile[:,-1]
            edges['w'] = tile[:,0]
            yield tk, edges, tile

def run(data):
    tiles = transform(data)
    edge_size = int(np.sqrt(len(tiles)))
    rest = list(tiles.keys())
    square = []
    square_generators = [generator(tiles, rest)]
    post_tiles = []
    def proc(pop_tk, edges):
        # curr
        square.append([pop_tk, edges['e'], edges['s']])
        post_tiles.append(tile)
        # next
        next_rest = rest.copy()
        [next_rest.remove(x[0]) for x in square]
        square_generators.append(generator(tiles, next_rest))
    def stopiter():
        square.pop()
        square_generators.pop()
        post_tiles.pop()


    while len(square)< len(tiles):
        i = len(square)
        if i == 0:
            pop_tk, edges, tile = next(square_generators[i])
            proc(pop_tk, edges)            
        elif i < edge_size:
            try:
                pop_tk, edges, tile = next(square_generators[i])
                if (edges['w'] == square[-1][1]).all():
                    proc(pop_tk, edges)   
            except StopIteration:
                stopiter()
        elif (i % edge_size)==0:
            try:
                pop_tk, edges, tile = next(square_generators[i])
                if (edges['n'] == square[i-edge_size][2]).all():
                    proc(pop_tk, edges)   
            except StopIteration:
                stopiter()
        else:
            try:
                pop_tk, edges, tile = next(square_generators[i])
                if (edges['w'] == square[-1][1]).all() & \
                        (edges['n'] == square[i-edge_size][2]).all():
                    proc(pop_tk, edges)
            except StopIteration:
                stopiter()
    a = np.array([x[0] for x in square]).reshape(edge_size,edge_size)
    return np.product([a[0,0], a[0,-1], a[-1,0], a[-1,-1]]), post_tiles

In [163]:
p1, post = run(t)
p1

29125888761511

## 2

In [164]:
def monster():
    return (np.array([list(x) for x in list('''                  # 
#    ##    ##    ###
 #  #  #  #  #  #   '''.splitlines())])=='#').astype(int)

def removeborder(post):
    s=np.sqrt(len(post)).astype(int)
    a=np.array([p[1:-1, 1:-1] for p in post])
    return np.concatenate(
        [np.concatenate(a[i:i+s], axis=1) for i in range(0,len(post),s)])

def run2(post):
    post = removeborder(post)
    es = len(post)
    for flud, rot in list(product([False, True],[0,1,2,3])):
        m = monster()
        if flud:
            m = np.flipud(m)
        m = np.rot90(m, rot)
        x,y = m.shape
        loc = []
        for i in range(x):
            for j in range(y):
                if m[i,j]==1:
                    loc.append((i,j))
        loc = np.array(loc)
        count = 0
        for row in range(es-x):
            for col in range(es-y):
                p = post[row:row+x, col:col+y]
                stt = True
                for l in loc:
                    if p[l[0], l[1]]!=1:
                        stt=False
                if stt:
                    count += 1
        if count>0:
            print(flud, rot, count, np.sum(post)-count*np.sum(m))

In [165]:
run2(post)

True 2 15 2219
